In [1]:
import pandas as pd 
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import faiss


/home/polina/miniforge3/envs/fixed_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-19 14:34:27.084360: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-19 14:34:27.270165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750332867.340826    7064 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750332867.360297    7064 cuda_blas.

In [2]:
df = pd.read_csv("movies_list.csv")

In [3]:
df.head()

,year,genre,director,time,page_url,movie_title,description,image_url,time_minutes
0,1994,драма,Фрэнк Дарабонт,1 час 48 минут,https://www.film.ru/movies/pobeg-iz-shoushenka,Побег из Шоушенка,Банкира Энди Дюфрейна обвиняют в убийстве жены...,https://www.film.ru/sites/default/files/styles...,108
1,1994,драма,Квентин Тарантино,2 часа 19 минут,https://www.film.ru/movies/kriminalnoe-chtivo,Криминальное чтиво,Знаковый фильм Квентина Тарантино в новом рема...,https://www.film.ru/sites/default/files/styles...,139
2,1991,драма,Джонатан Демме,1 час 58 минут,https://www.film.ru/movies/molchanie-yagnyat,Молчание ягнят,Психопат похищает и убивает молодых женщин по ...,https://www.film.ru/sites/default/files/styles...,118
3,2001,мультфильмы,Хаяо Миядзаки,2 часа 5 минут,https://www.film.ru/movies/unesennye-prizrakami,Унесенные призраками,Маленькая Тихиро вместе с мамой и папой переез...,https://www.film.ru/sites/default/files/styles...,125
4,1985,комедия,Роберт Земекис,1 час 48 минут,https://www.film.ru/movies/nazad-v-budushchee,Назад в будущее,"Марти МакФлай, типичный американский тинейджер...",https://www.film.ru/sites/default/files/styles...,108


In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2") 

/home/polina/miniforge3/envs/fixed_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
embeddings = model.encode(df['description'].tolist(),show_progress_bar=True)

Batches:   0%|          | 0/493 [00:00<?, ?it/s]

In [5]:
embeddings = np.array(embeddings).astype("float32")

In [6]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [20]:
# Запрос пользователя
user_query = "Мальчик волшебник с волшебной палочкой"
query_vector = model.encode([user_query],convert_to_numpy=True).astype("float32")

# Векторизуем запрос
#query_embedding = model.encode([user_query])

# Считаем схожесть запроса с каждым сериалом
#similarities = cosine_similarity(query_embedding, embeddings)[0]  # получаем 1D-массив

# Получим индексы топ-5 самых похожих сериалов
#top_n = 5
#top_indices = similarities.argsort()[-top_n:][::-1]  # сортировка по убыванию

# Выводим названия и описания похожих сериалов
# for i in top_indices:
#     print(f"\n🎬 Название: {df.iloc[i]['tvshow_title']}")
#     print(f"📄 Описание: {df.iloc[i]['description']}")
#     print(f"🔍 Схожесть: {similarities[i]:.3f}")

k = 9
distances, indices = index.search(query_vector, k)

# Выводим результаты
for i in indices[0]:
    print(f"\n🎬 Название: {df.iloc[i]['movie_title']}")
    print(get_wikipedia_summary(df.iloc[i]['movie_title'], year=None, lang='ru'))
    # print(f"📄 Описание: {df.iloc[i]['description']}")


🎬 Название: Шазам!
«Шаза́м!» (англ. Shazam!) — американский комедийный супергеройский фильм 2019 года режиссёра Дэвида Ф. Сандберга, основанный на одноимённом персонаже комиксов DC. Автором сценария выступил Генри Гейден, также работавший над сюжетом совместно с Дарреном Лемке. Кинокомикс является седьмым фильмом Расширенной вселенной DC. Главные роли исполнили Закари Ливай и Ашер Энджел; в фильме также снялись Марк Стронг, Джек Дилан Грейзер и Джимон Хонсу.
Проект находился в разработке с начала 2000-х годов, одн...

🎬 Название: Пацан против всех
«Пацан против всех» (англ. Boy Kills World) — художественный фильм режиссёра Морица Мора. Продюсируют фильм Сэм Рэйми и Рой Ли. Главные роли исполнили Билл Скарсгард, Джессика Рот и Эндрю Кодзи. Съёмки проходили в Южной Африке.
Мировая премьера фильма состоялась на Международном кинофестивале в Торонто 9 сентября 2023 года. В США фильм вышел 26 апреля 2024 года. В России фильм вышел 16 мая 2024 года....

🎬 Название: Приключения Реми
None

🎬 

In [8]:
np.save("movie_vectors.npy", embeddings)
faiss.write_index(index, "index.bin")

In [14]:
title = 'Шазам!'
year = '2019'

In [17]:
import wikipediaapi
from bs4 import BeautifulSoup
import re

def get_wikipedia_summary(title, year=None, lang='ru'):
    """
    Получение краткого содержания с Википедии
    с указанием пользовательского агента
    """
    # Указываем корректный User-Agent
    user_agent = "MovieRecommendationBot/1.0 (https://your-project-url.com; contact@example.com)"
    wiki_wiki = wikipediaapi.Wikipedia(
        language=lang,
        user_agent=user_agent
    )
    
    # Пытаемся найти страницу с уточнением года
    search_title = f"{title} (фильм, {year})" if year else title
    page = wiki_wiki.page(search_title)
    
    if not page.exists():
        # Пробуем без года
        page = wiki_wiki.page(title)
        if not page.exists():
            return None
    
    # Пытаемся получить раздел "Сюжет" через API
    section_titles = ["Сюжет", "Содержание", "Фабула", "Plot"]
    plot_section = None
    
    for section in section_titles:
        if section in page.sections:
            plot_section = page.section_by_title(section)
            if plot_section:
                return plot_section.text
    
    # Если раздел не найден, возвращаем краткое описание страницы
    
    return page.summary[:500] + "..."

In [19]:
get_wikipedia_summary(title, year=None, lang='ru')
len(get_wikipedia_summary(title, year=None, lang='ru'))

503

In [ ]:
# Кеш для хранения уже полученных сюжетов
plot_cache = {}

def recommend_movies_with_plot(movie_title, top_n=5, source='wikipedia'):
    if movie_title not in df["movie_title"].values:
        return "Фильм не найден в базе"
    
    idx = df[df["movie_title"] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    movie_indices = [i[0] for i in sim_scores]
    
    recommendations = df.iloc[movie_indices].copy()
    
    # Добавляем сюжеты для рекомендаций
    for i, row in recommendations.iterrows():
        title = row['movie_title']
        
        # Используем кеш, чтобы не делать запросы повторно
        if title not in plot_cache:
            if source == 'gigachat':
                plot_cache[title] = get_gigachat_summary(
                    title, 
                    row['director'],
                    row['year']
                )
            elif source == 'wikipedia':
                plot_cache[title] = get_wikipedia_summary(
                    title,
                    row['year']
                )
            else:
                plot_cache[title] = row['description']  # fallback
        
        recommendations.at[i, 'plot_summary'] = plot_cache[title]
    
    return recommendations[['movie_title', 'year', 'director', 'genre', 'plot_summary']]